In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [7]:



X_test_seq = np.load("../data/X_test_seq.npy")
Y_test_seq = np.load("../data/Y_test_seq.npy")


test_dataset = TimeSeriesDataset(X_test_seq, Y_test_seq)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
class LSTMModel(torch.nn.Module):
    def __init__(self, input_size=X_test_seq.shape[2], output_size=Y_test_seq.shape[1], hidden_size=512, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [12]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [ ]:
model = LSTMModel().to(device)
model.load_state_dict(torch.load("../models/neural_networks/lstm_model.pth"))


<All keys matched successfully>

In [14]:
import numpy as np

model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        predictions.append(outputs.cpu().numpy())
        actuals.append(targets.cpu().numpy())

# Concatenate results
predictions = np.concatenate(predictions, axis=0)
actuals = np.concatenate(actuals, axis=0)

RuntimeError: Input and parameter tensors are not at the same device, found input tensor at cuda:0 and parameter tensor at cpu

In [ ]:
from sklearn.metrics import r2_score

mse_per_sensor = np.mean((actuals - predictions) ** 2, axis=0)
for idx, loss in enumerate(mse_per_sensor):
    rmse = np.sqrt(loss)
    r2 = r2_score(actuals[:, idx], predictions[:, idx])
    print(f"Sensor {idx}: RMSE = {rmse:.4f}, R² = {r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt

sensor_idx = 0  # Change to visualize other sensors
plt.figure(figsize=(12, 6))
plt.plot(actuals[:, sensor_idx], label='Actual')
plt.plot(predictions[:, sensor_idx], label='Predicted')
plt.title(f'Sensor {sensor_idx} Prediction vs Actual')
plt.xlabel('Time Step')
plt.ylabel('Scaled Value')
plt.legend()
plt.show()